In [1]:
!pip --quiet install transformers datasets torch

In [2]:
import transformers
import datasets
from transformers.pipelines.pt_utils import KeyDataset
from pprint import PrettyPrinter
from tqdm.auto import tqdm

pprint = PrettyPrinter(compact=True).pprint

In [ ]:
data=[]
with open("sorted_qa_binary_parsebank.tsv", 'r') as f:
    for line in f:
        line=line.rstrip("\n")
        cols=line.split("\t")
        data.append(cols)

texts = [text[2] for text in data]
ids = [text[0] for text in data]

In [4]:
def load_dataset(name, files):
    dset = datasets.load_dataset(
        "csv",
        data_files={name: files},
        delimiter="\t",
        split=name, # so that it returns a dataset instead of datasetdict
        column_names=['id', 'label', 'text', 'prob1', 'prob2'],
        features=datasets.Features({    # Here we tell how to interpret the attributes
        "text":datasets.Value("string"),
        "label":datasets.ClassLabel(names=["QA_NEW", "NOT_QA"]),
        "id":datasets.Value("string"),
        "prob1":datasets.Value("float"),
        "prob2":datasets.Value("float"),
         }))
    return dset

In [5]:
dataset = load_dataset("train", "qa-sorted/sorted_qa_binary_parsebank.tsv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /users/annieske/.cache/huggingface/datasets/csv/default-3b78b2539888fc6b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [6]:
def replace_n(example):
  example["text"] = example["text"].replace("\n", " ")
  example["text"] = example["text"].replace("\\n", " ")

  return example

dataset = dataset.map(replace_n)

Map:   0%|          | 0/31655 [00:00<?, ? examples/s]

In [20]:
token_classifier = transformers.pipeline("ner", model="../models/token-qa/question_model", tokenizer="xlm-roberta-base", aggregation_strategy="simple", ignore_labels=[""],device=0) # make ignore labels empty to not ignore O
# simple aggregation has been best so far?
 #stride=50, 

In [21]:
tokens = token_classifier(KeyDataset(dataset, "text")) #texts instead of keydataset

In [22]:
for idx, extracted_entities in enumerate(tokens):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

Original text:
Olet täällä  31.1.2011  Kysymys  Terve, semmoista olisin kysynyt, että mitä tarkoittaa aivopoimut ja mihinkä se mahdollisesti vaikuttaa? Minun 4,5v tyttöni aivot on kuvattu ja lääkäri sanoi että mahdollisesti aivopoimuja voi olla liian vähän. Puheterapeutille oli lähete, mutta hänen mielestään artikuloi hyvin ja muutenkin ok..ei tarvitse sitä... Kaiken tämän ruljanssin takana on pieni pää tytölläni, jota on tutkittu kohta jo 3,5 vuotta ja selkoa ei ole tullut. Hänellä on todettu mikrokefalia, pään ympärys on tällä hetkellä 42,3 cm...että osaisitteko selittää niistä poimuista ja mihin ne mahdollisesti vaikuttaa? Lääkäri epäili että voi olla vaikeampaa oppia jotain asioita. Mutta olisi mukavaa jos voisitte jotain vastailla.  Vastaus  Hyvä kysyjä,  Noihin tutkimustuloksiin ei pidä tuijottaa. Pääasiahan on miten tyttärenne kehittyy. Jolleivät vanhemmat eivätkä edes puheterapeutti ole huomanneet mitään kuntoutettavaa, voitte mielestäni unohtaa löydökset. Jos jotain poikkeavaa

Original text:
Hakutulokset  Kysy allergiasta  Kysymys  Hei! Minulla on ollut noin 8 vuoden ajan noin 1-2 kuukauden välein joskus voi olla vähän enemmän väliä, mutta pääsääntöisesti aivastuskohtauksia, aivastan n. 10 tuntia yhtäjaksoisesti ja nenäliinoja kuluu 100 kpl paketti ja joskus enemmän, nenässä on semmoinen olo kun siellä kävelisi miljoona muurahaista,kun kohtaus on ohi sen jälkeen on kamala päänsärky ja ikeniä jomottaa ja silmät on ihan ummessa. Olen saanut lääkäriltä nenäsumutetta ja antihistamiinia, mutta niistä ei ole apua kun kohtaus alkaa yllättäin. En ole löytänyt mitään selitystä tälle kun se voi alkaa yks kaks ja missä ja milloinka vaan eli ei katso aikaa ei paikkaa, mutta raju "tauti" on, yleensä seuraava päivä menee leväten. Mitä pitäisi tehdä, mikä avuksi?  Vastaus  Voimakkaat, kivuliaat aivastuskohtaukset ovat harvinaisia. Niihin ei ole mitään yksittäistä syytä. Erikoislääkärillä käynti on ehdottomasti paikallaan. En ryhdy arvailemaan, mitä tutkimuksissa löytyy vai

Original text:
Vsd ja pieni koko  Kysymys  Hei, minua mietityttää että voiko sydänvika vaikuttaa vauvan kokoon? Meillä monen muuttuneen diagnoosin jälkeen todettiin vauvalla (joka on mahassa edelleen rv 34) VSD, ensin oli epäily hlhs, mutta kammio on kasvanut oikean kammion mukana, kapoisampana kylläkin, lääkärin mukaan kammion koko johtuu isosta vsd:tä. Kävimme juuri vauvan painoarviossa ja se oli vain 1764 g (rv 33+5), en siinä ymmärtänyt kysyä voiko sydänvika liittyä pieneen kokoon, kromosomit on tutkittu ja ne olivat kunnossa. Kuinka luotettava tutkimus se on? Niskaturvotusta vauvalla oli rv 12 0,6 eli ihan normaali.  Vastaus  Sydänvikainen tai yleensäkin sikiö, jolla on jokin rakennepoikkeavuus, ovat usein keskimääräistä pienempiä. Välttämättä näin ei kuitenkaan ole, vaan sikiö voi kasvaa aivan normaalisti. Kyseisen sikiön painoarvio menee melko lähelle normaalikäyrästön alarajaa ja vaattii siten tarkkaa kasvuseurantaa, mikä varmaan toteutuukin sydäntilanteen seurannan yhteydessä.

Original text:
Selaa kategorioita  Haava  Kysymys  Hei!  Olen nyt ymmärtänyt, että HIV ei siis tartu pintanaarmuista (esimerkiksi paperiviiltohaava, josta tulee verta), vaan haavan täytyy olla niin iso, että se vuotaa niin pahasti, että sitä täytyisi jo näyttää terveydenhoitajalle. Onko tämä totta? Ihmettelen, miksi kuitenkin laboratorioalalla kuitenkin tehdään altistuksesta tapaturmailmoitus, jos verta menee ihottumaiselle, haavaiselle tai rikkinäiselle iholle. Miksi näin, jos kuitenkaan HIV ei voi tästä tarttua? Vai voiko se tarttua?  Itselläni on paperiviiltohaavoja vähän väliä ja käsittelen näyteputkia lähes päivittäin. Pelkään HIV tartuntaa, ja on hämmentävää saada ristiriitaista tietoa. Onko se 100% varmaa, että mistään muusta haavasta HIV ei voi tarttua kuin syvästä terveydenhoitajan hoitoa vaativasta haavasta?  Vastaus  Hei. Hiv ei tartu paperiviiltohaavojen eikä pienten haavojen välityksellä. Työpaikoilla, joissa käsitellään verituotteita ja/tai neuloja on omat ohjeistuksensa 

Original text:
Vilkas sikiö (jatkoa)  Kysymys  Kävin tänään uusintaultrassa 21+6vko. Sikiön painoarvio oli 0,450 g ja päämitta vastasi viikkoja 22 ja raajat viikkoa 20. Kystaa ei näkynyt? Sovittiin raskauden jatkumisesta. Lapsivettä oli hiukan enemmän kuin normaalisti, napavirtaukset kaikki ok. Mistään löydöksistä ei mainittu enää? kasvot ok ja nenäkin ok. Ihmettelen enää tuota pään asiaa mansikkamaista, kun tänään sano sama lääkäri ettei ole mitään epämuodostumia??  Vastaus  Mansikkamainen pään muoto tarkoittaa, että pää on takaraivolta litteä ja siten mitta taka-eturaivo on tavallista lyhyempi. Tällainen pään muoto yhdistetään esim. 18-trisomiaan (yleensä muitakin löydöksiä).  Hieman tuntuu oudolta, että aikaisemmin oli runsaastikin erilaisia poikkevia löydöksiä (2 napasuonta, lyhyet luut, pleksuskysta jne) ja nyt siis kaikki normaalia. En pysty sanomaan mitään selitystä "kadonneille poikkeavuuksille". Kyllä asiaan pitää tutkineen lääkärin ottaa kantaa.  Kysymyksestä ei käy ilmi pitk

...

Original text:
Olet täällä  Ultra viikolla 12 + 2, sikiön niskaturvotus 3,3 mm  30.5.2006  Kysymys  Kävin juuri ultrassa viikolla 12+2 ja sikiön niskaturvotus oli 3,3 mm. Pelkään kovasti, että se on down. En kestä tätä odottamista. Mikä riski minulla on saada down? Ikäni on 27 v. Sikiön koko vastasi ultran mukaan tuota 12+2 viikkoa. Odotan pääsyä lapsivesipunktioon, kun asia vaivaa todella paljon. Masentaa!  Vastaus  Ikäriski 27-vuotiaalla raskausviikolla 12 on noin 1/800. 3,3 mm niskaturvotus nostaa sitä lähes 10-kertaiseksi eli riski on noin 1 prosentin luokkaa.  Keskustelu
Original id:
7-779921
Extracted entities:
{'entity_group': 'B', 'score': 0.9998907, 'word': 'Olet', 'start': 0, 'end': 4}
{'entity_group': 'I', 'score': 0.9999989, 'word': 'täällä Ultra viikolla 12 + 2, sikiön niskaturvotus 3,3 mm 30.5.2006 Kysymys Kävin juuri ultrassa viikolla 12+2 ja sikiön niskaturvotus oli 3,3 mm. Pelkään kovasti, että se on down. En kestä tätä odottamista. Mikä riski minulla on saada down? Ik

Original text:
Ketonen vastaa  saab 90, tai 9000 turbo  Kysymys:  onko mahdollista löytää suomesta saab 90 turbo ajokuntoinen ja hyvä. mitä olet mielta kannattaako ostaa saab 9000 3.0 t yli 200hv v6 koneella jolla ajettu yli300.000 jos auto pidetty hyvin. kulutus ja tulevat korjaukset  Tapio Ketonen:  Alkuperäistä tehdastekoista Saab 90 Turboa ei ole mahdollista ostaa, koska sellaisia ei rakennettu. Saab 900 Turboja varmaan vielä löytyy, siis niitä pitkittäismoottorisiakin. Ne ovat kuitenkin jo vähintään 20 vuotta vanhoja, joten kunnon kanssa on vähän niin ja näin.  Ysitonnisetkin ovat jo niin vanhoja, että niitä ei kannata ostaa kuin autoja oikein hyvin tuntevien. Koska kysyt asiaa täältä, vastaus kysymykseesi on: Ei kannata. Vanha Saab voisi olla kiinnostava kohde entisöidä, ja osiakin löytyy vielä helposti. Arvelen kuitenkin, että kysyt asiaa hankkiaksesi käyttöauton. Suosittelen katselemaan tähän tarkoitukseen uudempia ajokkeja.
Original id:
0-1540065
Extracted entities:
{'entity_g

Original text:
Selaa kategorioita  HIV-tartunnan mahdollisuus  Kysymys  Hei, lähetin jo kerran kysymykseni, mutta siihen ei ole tullut vastausta. Kokeilen siis uudestaan, jos viesti ei vaikka ole tullut perille. Ja jos vastaavaan kysymykseen ei vain ole ehditty vielä vastaamaan, pahoitteluni tästä toisesta viestistä.  Eli onko mahdollista saada verensokerin ottolaitteen lansetista (ei ole ontto) hiv- tai hepatiittitartuntaa? Ottolaitettani on mahdollisesti räplätty enkä älynnyt vaihtaa lansettia ennen kuin otin itseltäni verensokerin. Olen 1-tyypin diabeetikko. Mahdollisen toisen ihmisen käytön tai vahingon ja minun käyttöni välissä oli aikaa noin tunnin verran. Olisiko lansettiin jäänyt toisen veri jo kuivunut tartuttamattomaksi tässä ajassa, tai vaikka jo puolessa tunnissakin? Eli miten on hiv-tartunnan laita? Entä hepatiittien, jotka voivat tarttua myös kuivuneen veren välityksellä? Pitääkö siis olla huolissaan ja mennä testeihin, vai onko se aiheetonta?  Tämä on siinä mielessä erit

Original text:
Kysymys  Tervehdys! Onko mahdollista saada HIV-tartunta samaa nuuskatykkiä käyttäessä? Nuuska kun aiheuttaa noita pieniä haavaumia huuliin.  Vastaus  Hei! Hiv ei tartu nuuskatykin välityksellä.  suuseksi  Kysymys  Onko mahdollista saada tartunta jos verta menee peniksen limakalvoille suuseksissä?  Vastaus  Suuseksissä riski saada hiv-tartunta on osapuolella, joka saa spermaa tai emätineritettä suuhunsa. Mikäli syljen mukana on pieni määrä verta, ei tartuntariskiä synny. Toki teoreettinen riski tartuntaan on, jos suusta vuotaa peniksen limakalvoille runsaasti verta. Tämä ei kuitenkaan ole realistinen tilanne.  Hiv pelko  Kysymys  Elän ajoittain todella pahassa hiv pelossa, haluasin vastauksen lyhyeen vaikkakin vähän outoon kysymykseen. Kärsin ajoittain voimakkaasta takapolen kutinasta ja hikoilusta joten voinko saada hiv tartunnan omista likaisista ja hikisistä alushousuista? kun on tapana joskus vaistomaisesti raapia takapuolta kutinan takia mikä on aika kiusallista.  Va

KeyboardInterrupt: 

In [12]:
from tqdm import tqdm

texts = dataset["text"]

In [13]:
questions = []
answers = []
for idx, extracted_entities in enumerate(tqdm(tokens)):
  previous = ""
  question = ""
  answer = ""
  # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
  # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
  for entity in extracted_entities:
    if entity["entity_group"] == "QB":
      question = question + dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
      previous = "QB"
      # I should take into account that sometimes QI might be before QB but maybe this is more trustworthy
    elif previous == "QB" and entity["entity_group"] == "QI" or previous == "QI" and entity["entity_group"] == "QB":
      question = question + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "QI"
    elif entity["entity_group"] == "AB":
      answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "AB"
    if previous == "AB" and entity["entity_group"] == "AI" or previous == "AI" and entity["entity_group"] == "AB":
      answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "AI"
    elif entity["entity_group"] == "O":
      previous = "O"
  # if answer == "" or question == "":
  #   continue
  questions.append(question)
  answers.append(answer)

  1%|          | 314/31655 [00:04<07:07, 73.31it/s]


KeyboardInterrupt: 

In [17]:
questions = []
for idx, extracted_entities in enumerate(tqdm(tokens)):
  previous = ""
  question = ""
  # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
  # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
  for entity in extracted_entities:
    if entity["entity_group"] == "QB":
      question = question + dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
      previous = "QB"
      # I should take into account that sometimes QI might be before QB but maybe this is more trustworthy
    elif previous == "QB" and entity["entity_group"] == "QI" or previous == "QI" and entity["entity_group"] == "QB":
      question = question + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "QI"
    elif entity["entity_group"] == "O":
      previous = "O"
  questions.append(question)

  1%|          | 250/31655 [00:03<07:09, 73.12it/s]


KeyboardInterrupt: 

In [18]:
answers = []
for i in range(len(questions)):
    answers.append("")

In [ ]:
answers = []
for idx, extracted_entities in enumerate(tqdm(tokens)):
  previous = ""
  answer = ""
  # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
  # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
  for entity in extracted_entities:
    elif entity["entity_group"] == "AB":
      answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "AB"
    if previous == "AB" and entity["entity_group"] == "AI" or previous == "AI" and entity["entity_group"] == "AB":
      answer = answer + dataset[idx]["text"][entity["start"]:entity["end"]]
      previous = "AI"
    elif entity["entity_group"] == "O":
      previous = "O"
  answers.append(answer)

In [19]:
for t, q, a in zip(texts, questions, answers):
  print("TEXT:", t)
  print("QUESTION:", q)
  print("ANSWER:", a)
  print("-------------")

TEXT: Olet täällä  31.1.2011  Kysymys  Terve, semmoista olisin kysynyt, että mitä tarkoittaa aivopoimut ja mihinkä se mahdollisesti vaikuttaa? Minun 4,5v tyttöni aivot on kuvattu ja lääkäri sanoi että mahdollisesti aivopoimuja voi olla liian vähän. Puheterapeutille oli lähete, mutta hänen mielestään artikuloi hyvin ja muutenkin ok..ei tarvitse sitä... Kaiken tämän ruljanssin takana on pieni pää tytölläni, jota on tutkittu kohta jo 3,5 vuotta ja selkoa ei ole tullut. Hänellä on todettu mikrokefalia, pään ympärys on tällä hetkellä 42,3 cm...että osaisitteko selittää niistä poimuista ja mihin ne mahdollisesti vaikuttaa? Lääkäri epäili että voi olla vaikeampaa oppia jotain asioita. Mutta olisi mukavaa jos voisitte jotain vastailla.  Vastaus  Hyvä kysyjä,  Noihin tutkimustuloksiin ei pidä tuijottaa. Pääasiahan on miten tyttärenne kehittyy. Jolleivät vanhemmat eivätkä edes puheterapeutti ole huomanneet mitään kuntoutettavaa, voitte mielestäni unohtaa löydökset. Jos jotain poikkeavaa ilmenee,